First we have to load the required libraries.

In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, AutoTokenizer
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import os
import openai

In [2]:
torch.manual_seed(42)

# Change tokenizer if needed
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Load finetuned model
# Change model_path to the directory of finetuned model
model_path = 'Neo-50K-100%-v2/checkpoint-44384'
model = AutoModelForCausalLM.from_pretrained(model_path, pad_token_id=tokenizer.eos_token_id).cuda()

In [3]:
device=torch.device("cuda")
print(device)

cuda


In [23]:
# Load Dataset
data_file = pd.read_csv('equal_dataset_35000.csv')
data_file.drop('Unnamed: 0', inplace=True, axis=1)
data_file

,text
0,<cs.CR>: The evolution of the traditional po...
1,<cs.CR>: Energy storage provides an effectiv...
2,<cs.CR>: In this paper we propose a novel me...
3,<cs.CR>: Hardware and software of secured em...
4,<cs.CR>: Today is the era of Internet of Thi...
...,...
34995,<physics.optics>: The paper introduces a new...
34996,<physics.optics>: A wave packet undergoes a ...
34997,<physics.optics>: The central theme of this ...
34998,<physics.optics>: Limited by the difficulty ...


In [27]:
def generate_text(model, idx_data, len_prompt, max_seq_len, do_sample=True, temperature=None, top_k=None):
    prompt = ' '.join(data_file['text'][idx].split()[1:1+len_prompt])

    input_ids = tokenizer(prompt, return_tensors='pt').to(device)

    if temperature:
        gen_text = model.generate(**input_ids, max_length=max_seq_len, do_sample=do_sample, temperature=temperature)
    elif top_k:
        gen_text = model.generate(**input_ids, max_length=max_seq_len, do_sample=do_sample, top_k=top_k)
    
    gen_text = tokenizer.decode(gen_text[0], skip_special_tokens=True)

    print('=================================Prompt=================================')
    print(prompt)
    print('\n')
    print('===========================Ground Truth Text===========================')
    print(' '.join(data_file['text'][idx].split()[1:1+max_seq_len]))
    print('\n')
    print('=============================Generated Text============================')
    print(gen_text)


In [30]:
# Temperature-based sampling
generate_text(model=model, idx_data=0, len_prompt=10, max_seq_len=100, temperature=0.7)

=================================Prompt=================================
The evolution of the traditional power system towards the modern


===========================Ground Truth Text===========================
The evolution of the traditional power system towards the modern smart grid has posed many new cybersecurity challenges to this critical infrastructure. One of the most dangerous cybersecurity threats is the False Data Injection (FDI) attack, especially when it is capable of completely bypassing the widely deployed Bad Data Detector of State Estimation and interrupting the normal operation of the power system. Most of the simulated FDI attacks are designed using simplified linearized DC model while most industry standard State Estimation systems are based on the nonlinear AC model. In this paper, a comprehensive FDI attack scheme is presented based


=============================Generated Text============================
The evolution of the traditional power system towards the

In [31]:
# Deterministic (argmax) sampling
generate_text(model=model, idx_data=0, len_prompt=10, max_seq_len=100, do_sample=False, temperature=1)

=================================Prompt=================================
The evolution of the traditional power system towards the modern


===========================Ground Truth Text===========================
The evolution of the traditional power system towards the modern smart grid has posed many new cybersecurity challenges to this critical infrastructure. One of the most dangerous cybersecurity threats is the False Data Injection (FDI) attack, especially when it is capable of completely bypassing the widely deployed Bad Data Detector of State Estimation and interrupting the normal operation of the power system. Most of the simulated FDI attacks are designed using simplified linearized DC model while most industry standard State Estimation systems are based on the nonlinear AC model. In this paper, a comprehensive FDI attack scheme is presented based


=============================Generated Text============================
The evolution of the traditional power system towards the

In [32]:
# Top_k sampling
generate_text(model=model, idx_data=0, len_prompt=10, max_seq_len=100, do_sample=True, top_k=40)

=================================Prompt=================================
The evolution of the traditional power system towards the modern


===========================Ground Truth Text===========================
The evolution of the traditional power system towards the modern smart grid has posed many new cybersecurity challenges to this critical infrastructure. One of the most dangerous cybersecurity threats is the False Data Injection (FDI) attack, especially when it is capable of completely bypassing the widely deployed Bad Data Detector of State Estimation and interrupting the normal operation of the power system. Most of the simulated FDI attacks are designed using simplified linearized DC model while most industry standard State Estimation systems are based on the nonlinear AC model. In this paper, a comprehensive FDI attack scheme is presented based


=============================Generated Text============================
The evolution of the traditional power system towards the